In [2]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

# Define the problem as a multi-objective optimization problem
creator.create("FitnessMulti", base.Fitness, weights=[1.0, -1.0])
creator.create("Individual", list, fitness=creator.FitnessMulti)

# Define the problem-specific parameters
n = 5  # Number of physical machines
num_generations = 50
s = 20  # Number of services
# Define the objective functions with alpha and beta
alpha = 0.5  # Weight for latency
beta = 0.5   # Weight for energy consumption

# Define parameters. This will be loaded from the database.
# Here, we use random values for demonstration purposes.
# Replace these with actual data when implementing in a real system.

# Variable number of virtual machines for each physical machine
num_virtual_machines = [random.randint(1, 10) for _ in range(n)]

################ Physical and virtual machine configurations ##################

# Randomly generate other parameters (replace with actual data)
R = np.random.uniform(0, 1, size=(n, max(num_virtual_machines)))  # Request network delay of the machine
I = np.random.uniform(0, 1, size=(n, max(num_virtual_machines)))  # IPS currently executed
X = np.random.uniform(0, 1, size=(n, max(num_virtual_machines)))  # Maximum IPS possible
S = np.random.uniform(0, 1, size=(n, max(num_virtual_machines)))  # Response network delay of the machine

PI = np.random.uniform(0, 1, size=(n, max(num_virtual_machines)))  # Power consumption in idle
PM = np.random.uniform(0, 1, size=(n, max(num_virtual_machines)))  # Maximum power consumption

# Calculate energy consumption based on the parameters
E = PI + (PM - PI) * (I / X)

################ Service configurations ###################

# Acceptable latencies for services
SAL = np.random.uniform(0, 1, size=s)
# Initialize IPS values for services (replace with actual data)
SIPS = np.random.uniform(0, 1, size=s)

# Define the objective functions
def evaluate_individual(individual):
    total_latency = 0.0
    total_energy = 0.0
    for i in range(s):
        p = individual[i]  # Physical machine where service is placed
        if p < n:
            v = random.randint(0, num_virtual_machines[p] - 1)  # Virtual machine on that physical machine
            if v < num_virtual_machines[p]:
                if (R[p][v] + (I[p][v] + SIPS[i]) / X[p][v] + S[p][v]) <= SAL[i]:
                    total_latency += R[p][v] + (I[p][v] + SIPS[i]) / X[p][v] + S[p][v]
                    total_energy += E[p][v]  # Consider energy consumption
    # Calculate the weighted sum of objectives
    weighted_latency = alpha * total_latency
    weighted_energy = beta * total_energy
    return np.array([weighted_latency, weighted_energy])

# Create a DEAP toolbox and register functions
toolbox = base.Toolbox()
toolbox.register("attr_int", random.randint, 0, n - 1)
toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.attr_int,), n=s)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Evaluation function
toolbox.register("evaluate", evaluate_individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=n - 1, indpb=0.2)
toolbox.register("select", tools.selBest)

# Crossover, mutation, and selection registration

# In the main loop, consider both latency and energy in the fitness function

if __name__ == "__main__":
    # Create the initial population
    population = toolbox.population(n=100)

    # Create statistics object to track performance
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min_lat", min)
    stats.register("min_energy", min)

    # Create a logbook to log statistics
    logbook = tools.Logbook()
    logbook.header = "gen", "evals", "min_lat", "min_energy"

    # Run the optimization using a basic Genetic Algorithm (GA)
    algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=num_generations, stats=stats,
                        halloffame=None, verbose=True)

    # Extract the Pareto front solutions (not necessarily Pareto-optimal in a basic GA)
    pareto_front = population

    # Print Pareto front solutions
    print("Pareto Front Solutions:")
    for ind in pareto_front:
        print("Latency:", ind.fitness.values[0])  # Extract the first element
        print("Energy Consumption:", ind.fitness.values[1])  # Extract the second element
        print("Placement:", ind)

gen	nevals	min_lat   	min_energy
0  	100   	(0.0, 0.0)	(0.0, 0.0)
1  	64    	(0.0, 0.0)	(0.0, 0.0)
2  	76    	(0.0, 0.0)	(0.0, 0.0)
3  	69    	(0.0, 0.0)	(0.0, 0.0)
4  	70    	(0.0, 0.0)	(0.0, 0.0)
5  	81    	(0.0, 0.0)	(0.0, 0.0)
6  	74    	(0.0, 0.0)	(0.0, 0.0)
7  	71    	(0.0, 0.0)	(0.0, 0.0)
8  	81    	(0.0, 0.0)	(0.0, 0.0)
9  	75    	(0.0, 0.0)	(0.0, 0.0)
10 	79    	(0.0, 0.0)	(0.0, 0.0)
11 	77    	(0.0, 0.0)	(0.0, 0.0)
12 	80    	(0.0, 0.0)	(0.0, 0.0)
13 	68    	(0.0, 0.0)	(0.0, 0.0)
14 	77    	(0.0, 0.0)	(0.0, 0.0)
15 	70    	(0.0, 0.0)	(0.0, 0.0)
16 	76    	(0.0, 0.0)	(0.0, 0.0)
17 	87    	(0.0, 0.0)	(0.0, 0.0)
18 	76    	(0.0, 0.0)	(0.0, 0.0)
19 	80    	(0.0, 0.0)	(0.0, 0.0)
20 	71    	(0.0, 0.0)	(0.0, 0.0)
21 	78    	(0.0, 0.0)	(0.0, 0.0)
22 	81    	(0.0, 0.0)	(0.0, 0.0)
23 	72    	(0.0, 0.0)	(0.0, 0.0)
24 	77    	(0.0, 0.0)	(0.0, 0.0)
25 	72    	(0.0, 0.0)	(0.0, 0.0)
26 	76    	(0.0, 0.0)	(0.0, 0.0)
27 	76    	(0.0, 0.0)	(0.0, 0.0)
28 	79    	(0.0, 0.0)	(0.0, 0.0)
29 	75    

/Library/Python/3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Library/Python/3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
